In [6]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
api_key = "e30eb1259555481b91906de4cb5a3cd5"
azure_endpoint = "https://artirst-assistant.openai.azure.com/"
api_version = "2024-02-15-preview"
llm = AzureOpenAI(
    model="gpt-3.5-turbo-instruct",
    deployment_name="35instruct",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="emb3large",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [8]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [9]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import FlatReader, HTMLTagReader
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader("test_data/")

docs = reader.load_data()

In [6]:
print(len(docs))

1


In [7]:
docs

[Document(id_='ece0de1c-27df-4aa7-944f-554e783e794a', embedding=None, metadata={'file_path': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_name': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_type': 'text/html', 'file_size': 2626, 'creation_date': '2024-03-06', 'last_modified_date': '2024-03-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='<div>\n  <h1 class="c-ttl-article__txt">結婚12年目「倖田來未」が明かす“夫と息子”との私生活\u3000「親子でプロレスにハマってます」</h1>\n  <p>\n    今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロかっこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆\n    Live Package「WINGS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与えているかをインタビューした。\n  </p>\n  <p>\n    倖田來未が1月18

In [8]:
# Indexing documentation 
index = VectorStoreIndex.from_documents(docs)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"


In [12]:
query_engine = index.as_query_engine()
response = query_engine.query("倖田來未さんは結婚して何年目ですか？")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
12年目


In [17]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is Koda Kumi and how long has she been in the music industry?")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"

Koda Kumi is a 40-year-old singer who has been in the music industry for 23 years.


In [16]:
query_engine = index.as_query_engine()
response = query_engine.query("倖田來未とは何者で、音楽業界でどれくらい働いていますか?")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"

倖田來未は歌手であり、今年でデビュー23年目を迎えています。


---

### Test Part 2: Deeper

In [24]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [25]:
api_key = "a83e2f795b9f4aada15fba7f6594d216"
azure_endpoint = "https://artist-assistant-uk-south.openai.azure.com/"
api_version = "2024-02-15-preview"
llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="35turbo16k",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ada2",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [26]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [27]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler

from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader("test_data/")

docs = reader.load_data()


pipeline = IngestionPipeline(
    transformations=[
        HTMLNodeParser.from_defaults(),
        SemanticSplitterNodeParser(
            buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
        ),
        embed_model,
    ],
)
nodes = pipeline.run(documents=docs)

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-s

In [28]:
print(len(nodes))

2


In [29]:
for node in nodes: 
    print(node)
    print("-----------------------")

Node ID: f73f756a-917b-47d2-b263-1453265c31e2
Text: 結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスにハマってます」
-----------------------
Node ID: a4ece099-72c5-4859-a9c2-2ed4ccb14ee4
Text: 今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロか
っこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆     Live Package「WIN
GS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与
えているかをインタビューした。 倖田來未が1月18日に発売したMusic＆Live     Package「WINGS」のメイントラックとな
るミディアムバラード「Wings」のクレジットを見ると、作曲者はHi－yunk（ハイユンク）とある。これは、倖田の夫であるKENJI03の作
家名義...
-----------------------


In [30]:
# Indexing 
index = VectorStoreIndex(nodes)

In [31]:
# Store database
from llama_index.core import StorageContext, load_index_from_storage

index.storage_context.persist(persist_dir="database/")

In [13]:
# load database 
# from llama_index.core import StorageContext, load_index_from_storage

# # rebuild storage context
# storage_context = StorageContext.from_defaults(persist_dir="<persist_dir>")

# # load index
# index = load_index_from_storage(storage_context)


In [32]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


query_engine = index.as_query_engine()
response = query_engine.query(
    "倖田來未さんは結婚して何年目ですか？"
)
print(response)

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-as

In [33]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("倖田來未さんは結婚して何年目ですか？")
print(response)

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-as

In [39]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='f73f756a-917b-47d2-b263-1453265c31e2', embedding=None, metadata={'tag': 'h1', 'file_path': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_name': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_type': 'text/html', 'file_size': 2626, 'creation_date': '2024-03-06', 'last_modified_date': '2024-03-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4c6ce9c3-c849-4146-9fc1-20a65f9396c0', node_type=<ObjectType.TEXT: '1'>, metadata={'tag': 'h1', 'file_path': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_name': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_type': 't

In [41]:
print(type(response.source_nodes))

<class 'list'>


In [44]:
for source in response.source_nodes: 
    print(source.text)
    print("________________________")

結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスにハマってます」
________________________
今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロかっこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆
    Live Package「WINGS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与えているかをインタビューした。
倖田來未が1月18日に発売したMusic＆Live
    Package「WINGS」のメイントラックとなるミディアムバラード「Wings」のクレジットを見ると、作曲者はHi－yunk（ハイユンク）とある。これは、倖田の夫であるKENJI03の作家名義だ。
「コンペで楽曲を集め、ラスト3曲に絞った上で、タイアップ先が『これがいい』と選んでくださったのが『Wings』でした。この曲に決まった時は、素直にうれしかったですね。私の大好きな曲だから。旦那は『倖田來未の新しい声を引き出したい』といつも考えているような人なので、今だからこそ私が歌える楽曲を、ということで完成させた曲なんです」
倖田は2011年12月3日、ロックバンド「BACK－ON」のKENJI03との結婚を発表。同月16日には妊娠中であることも公表した。2 人の仲を取り持ったのは、妹のmisonoだったという。倖田は夫とのなれそめをこう振り返る。
「妹のミーモ（misonoの愛称）が当時、旦那とコラボしていて、ミーモにコラボ曲の入ったアルバムを聞かせてもらったんです。『えっ、コラボしている人、めっちゃいい声してるやん』というのが、私の第一印象。まず声に惚れました（笑）。その後、ミュージックビデオを見ると『えっ、イケメンやん』と。当時の私は夜に出歩かない時期が半年以上も続いていました。ミーモは心配して『お姉もそろそろ、ご飯を食べにとか、外に出かけた方がいいよ』と言ってくれていました。それで、外出した時に彼を紹介してもらいました」
________________________


In [48]:
!pip install llama-index-llms-openai

In [49]:
!pip install llama-index

In [56]:
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.7/491.7 kB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 3.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 3.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.7 MB/s eta 0:00:00


In [65]:
import nest_asyncio
nest_asyncio.apply()

In [72]:
from llama_index.core.evaluation import DatasetGenerator

# Generate Question
data_generator = DatasetGenerator.from_documents(docs)

/home/dngback/anaconda3/envs/artistAssistant/lib/python3.12/site-packages/llama_index/core/evaluation/dataset_generation.py:212: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [74]:
eval_questions = data_generator.generate_questions_from_nodes()

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"


/home/dngback/anaconda3/envs/artistAssistant/lib/python3.12/site-packages/llama_index/core/evaluation/dataset_generation.py:309: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [76]:
print(eval_questions)

['Who is the artist mentioned in the context information?', 'How long has the artist been in the music industry?', "What is the title of the main track in the artist's latest release?", 'Who is the composer of the main track?', 'When did the artist announce her marriage?', 'Who played a role in bringing the artist and her husband together?', "What was the artist's first impression of her husband?", 'Why was the artist not going out at night during a certain period?', 'Who expressed concern for the artist and encouraged her to go out?', 'How did the artist meet her husband?']


In [77]:
import pandas as pd
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response


# define jupyter display function
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": (
                response.source_nodes[0].node.get_content()[:1000] + "..."
            ),
            "Evaluation Result": eval_result,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [78]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator


evaluator_gpt4 = RelevancyEvaluator(llm=llm)

In [79]:
query_engine = index.as_query_engine()
response_vector = query_engine.query(eval_questions[1])
eval_result = evaluator_gpt4.evaluate_response(
    query=eval_questions[1], response=response_vector
)

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-as

In [81]:
display_eval_df(eval_questions[1], response_vector, eval_result)

ValueError: Length of values (10) does not match length of index (1)